<h1 style="color: green;">Summary</h1>
<p style="color: black;">
In this section data is pulled from the database and exported to csv format ready for exploratory data analysis and feature engineering
</p>


<h1 style="color: green;">Importing libraries</h1>

In [3]:
import os

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# SQLAlchemy==1.4.35
import sqlalchemy

# environment variables
from dotenv import load_dotenv


In [4]:
# Loading environment variables
load_dotenv()

True

<h1 style="color: green;">Connecting to database</h1>

In [5]:
sqlUrl = sqlalchemy.engine.URL.create(
    drivername = os.getenv('drivername'),
    username = os.getenv('username'),
    password = os.getenv('password'),
    host = os.getenv('host'),
    port = os.getenv('port'),
    database = os.getenv('database')
)

engine = sqlalchemy.create_engine(sqlUrl)

<h3 style="color: green;">Reading data</h3>

In [6]:
# Reading receipt table

Receipt = pd.read_sql_table("receipt", engine)

print(Receipt.shape)
Receipt.tail()


(903, 10)


,Receipt_id,Venue_id,Total_Nbr_of_Items,Total_Price,Receipt_Nbr,Trans_number,Barcode,Receipt_Date,Receipt_Time,Date_Added
898,909,53,1,17.99,9032023223000,0,0,2023-03-09,22:30:00,2023-05-20 11:49:57
899,910,53,1,15.99,28032023115500,0,0,2023-03-28,11:55:00,2023-05-20 11:53:44
900,911,3,1,1.80,21052023121600,0,0,2023-05-21,12:16:00,2023-05-21 14:09:00
901,912,53,1,10.99,23052023101400,0,0,2023-05-23,10:14:00,2023-05-23 10:30:49
902,913,36,16,18.67,26052023161414,0,0,2023-05-26,16:14:14,2023-05-26 19:47:03


In [8]:
# Reading in Venue details
Venue_details = pd.read_sql_table("venue_details", engine)

Venue_details[['Venue_id','Venue']].head()

,Venue_id,Venue
0,1,Aldi
1,2,B&Q
2,3,Wilko
3,4,Sports direct.com
4,5,Europe


In [9]:
# Reading payment table
Payment = pd.read_sql_table("payment", engine)
Payment[['Payment_id','Receipt_id','Payment_Type','Card_Source','Date_Added']].head()

,Payment_id,Receipt_id,Payment_Type,Card_Source,Date_Added
0,1,1,Card,Contactless,2020-07-23 18:34:35
1,2,2,Card,Contactless,2020-07-23 20:29:03
2,3,3,Card,Contactless,2020-07-23 20:54:53
3,4,4,Card,Contactless,2020-07-23 21:15:05
4,5,5,Card,Contactless,2020-07-23 21:32:10


In [10]:
# Reading item table
Item = pd.read_sql_table("item", engine)
Item.tail()

,Item_id,Venue_id,Receipt_id,Venue_Item_code,Item_name,Item_Price,Date_Added
5067,5128,36,913,0,Chicken soup,0.45,2023-05-26 19:47:03
5068,5129,36,913,0,Chicken soup,0.45,2023-05-26 19:47:03
5069,5130,36,913,0,Brown onions,1.10,2023-05-26 19:47:03
5070,5131,36,913,0,Chicken thighs,3.14,2023-05-26 19:47:03
5071,5132,36,913,0,Chicken thighs,3.14,2023-05-26 19:47:03


<h3 style="color: green;">Merging the tables to create the raw data</h3>

In [11]:
# Merging Venue_details with Receipt to retrieve Venue
Receipt = pd.merge(Receipt, 
                   Venue_details[['Venue_id','Venue']], 
                   on='Venue_id', how='left')
Receipt.head()

,Receipt_id,Venue_id,Total_Nbr_of_Items,Total_Price,Receipt_Nbr,Trans_number,Barcode,Receipt_Date,Receipt_Time,Date_Added,Venue
0,1,1,8,6.78,1488772027005024,0,0,2020-05-28,11:09:45,2020-07-23 18:25:38,Aldi
1,2,1,9,6.32,1173772027004007,0,0,2020-06-24,12:57:56,2020-07-23 20:25:36,Aldi
2,3,1,9,7.44,6880772027011805,0,0,2020-07-01,14:28:27,2020-07-23 20:51:18,Aldi
3,4,1,12,8.27,7450772027005024,0,0,2020-06-10,13:07:51,2020-07-23 21:13:19,Aldi
4,5,1,11,8.10,3851772027003031,0,0,2020-05-25,11:31:37,2020-07-23 21:30:04,Aldi


In [12]:
# merging Payment to Receipt for the analysis
Receipt_Payment = pd.merge(Receipt, 
                           Payment[['Receipt_id','Payment_Type','Card_Source']], 
                           on='Receipt_id', how='left')
Receipt_Payment.head()

,Receipt_id,Venue_id,Total_Nbr_of_Items,Total_Price,Receipt_Nbr,Trans_number,Barcode,Receipt_Date,Receipt_Time,Date_Added,Venue,Payment_Type,Card_Source
0,1,1,8,6.78,1488772027005024,0,0,2020-05-28,11:09:45,2020-07-23 18:25:38,Aldi,Card,Contactless
1,2,1,9,6.32,1173772027004007,0,0,2020-06-24,12:57:56,2020-07-23 20:25:36,Aldi,Card,Contactless
2,3,1,9,7.44,6880772027011805,0,0,2020-07-01,14:28:27,2020-07-23 20:51:18,Aldi,Card,Contactless
3,4,1,12,8.27,7450772027005024,0,0,2020-06-10,13:07:51,2020-07-23 21:13:19,Aldi,Card,Contactless
4,5,1,11,8.10,3851772027003031,0,0,2020-05-25,11:31:37,2020-07-23 21:30:04,Aldi,Card,Contactless


In [13]:
# merging item to Receipt_Payment
raw0 = pd.merge(Receipt_Payment,
                     Item[['Receipt_id','Item_id','Item_name','Item_Price']], 
                     on='Receipt_id', 
                     how='left' )

In [14]:
raw0.head()

,Receipt_id,Venue_id,Total_Nbr_of_Items,Total_Price,Receipt_Nbr,Trans_number,Barcode,Receipt_Date,Receipt_Time,Date_Added,Venue,Payment_Type,Card_Source,Item_id,Item_name,Item_Price
0,1,1,8,6.78,1488772027005024,0,0,2020-05-28,11:09:45,2020-07-23 18:25:38,Aldi,Card,Contactless,1,Bloomer Seed 800G,0.99
1,1,1,8,6.78,1488772027005024,0,0,2020-05-28,11:09:45,2020-07-23 18:25:38,Aldi,Card,Contactless,2,Cheese Singles,0.75
2,1,1,8,6.78,1488772027005024,0,0,2020-05-28,11:09:45,2020-07-23 18:25:38,Aldi,Card,Contactless,3,Cheese Spread,0.79
3,1,1,8,6.78,1488772027005024,0,0,2020-05-28,11:09:45,2020-07-23 18:25:38,Aldi,Card,Contactless,4,Ham cooked lean,1.39
4,1,1,8,6.78,1488772027005024,0,0,2020-05-28,11:09:45,2020-07-23 18:25:38,Aldi,Card,Contactless,5,Eggs caged 15pk,1.18


In [15]:
raw0.shape

(5072, 16)

<h1 style="color: green;">Exporting raw data</h1>

In [16]:
raw0.to_csv("../Data/homeshopping.csv",index=False)